In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import re
import numpy as np

Um an meinen Datensatz zu kommen, muss ich alle Transferfenster seit der Saison 2003/04 bis heute scrapen. Ich beginne zunächst mit dem ersten:

In [2]:
my_url1 = 'https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2003&s_w=&leihe=0&intern=0'

Bei der im Kurs gelernten Request-Abfrage der URL kam ein 404-Error. Deshalb habe ich folgende Vorgehensweise gegoogelt:

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

In [4]:
req = urllib.request.Request(url = my_url1, headers = headers)

In [5]:
with urllib.request.urlopen(req) as response:
   page_html = response.read()

Damit hat es dann geklappt, ich hole mir die Soup für die erste Seite:

In [6]:
soup = BeautifulSoup(page_html,"html.parser")

Was brauche ich von den jeweiligen Seiten/Soups?
- Transferfenster/Saison
- Abgebender Verein (CH)
- ID des abgebenden Vereins (CH)
- Spielername
- Aufnehmender Verein (Ausland)
- Liga des aufnehmenden Vereins (leider nicht möglich, nur Flagge des Landes)
- ID des aufnehmenden Vereins (Ausland) (um später dem Klub die aktuelle Liga zuordnen zu können.)
- Ablösesumme (Brauche ich nicht dringend, könnte aber spannend sein.)

So komme ich an den ersten Klubnamen heran:

In [7]:
soup.find_all('h2', {'class':'content-box-headline content-box-headline--inverted content-box-headline--logo'})[0].img['alt']

'Grasshopper Club Zürich'

Mit diesem For-Loop komme ich an die zehn abgebenden Super-League-Vereine ran:

In [8]:
clubs1 = []

data_club1 = soup.find_all('h2', {'class':'content-box-headline content-box-headline--inverted content-box-headline--logo'})
for club1 in data_club1:
    clubs1.append(club1.img['alt'])
    
clubs1

['Grasshopper Club Zürich',
 'FC Basel 1893',
 'Neuchâtel Xamax',
 'BSC Young Boys',
 'FC Zürich',
 'Servette FC',
 'FC Thun',
 'FC Wil 1900',
 'FC Aarau',
 'FC St. Gallen 1879']

So komme ich an die Klub-ID des ersten, abgebenden Vereins heran:

In [9]:
soup.find_all('h2', {'class':''})[0].a['href'].split('-')[1]

'504'

Mit diesem For-Loop hole ich mir die Club-ID des abgebenden Vereins:

In [10]:
club_ids1 = []

data_club_id1 = soup.find_all('h2', {'class':''})
for club_id1 in data_club_id1:
    club_ids1.append(club_id1.a['href'].split('-')[1])
club_ids1

['504', '26', '625', '452', '260', '61', '938', '850', '116', '257']

So suche ich die Transfer-Saison raus:

In [11]:
soup.find('h1', {'class':'content-box-headline'}).text.strip().split(' ')[1]

'03/04'

Spielernamen:

In [12]:
soup.find_all('table')[2].find_all('tr')[1].find('td').find('div', {'class':'di nowrap'}).a['title']

'Baykal Bellusci'

Spieler_ID:

In [13]:
soup.find_all('table')[2].find_all('tr')[1].find('td').find('div', {'class':'di nowrap'}).a['href'].split('/')[4]

'2873'

Das Alter:

In [14]:
soup.find_all('table')[2].find_all('tr')[1].find('td', {'class': 'zentriert alter-transfer-cell'}).text

'20'

Die Nationalität des Spielers:

In [15]:
soup.find_all('table')[2].find_all('tr')[1].find('td', {'class': 'zentriert nat-transfer-cell'}).img['alt']

'Türkei'

In [16]:
soup.find_all('table')[2].find_all('tr')[1].find('td', {'class': 'zentriert nat-transfer-cell'})

<td class="zentriert nat-transfer-cell"><img alt="Türkei" class="flaggenrahmen" src="https://tmssl.akamaized.net/images/flagge/tiny/174.png?lm=1520611569" title="Türkei"/><br/><img alt="Schweiz" class="flaggenrahmen" src="https://tmssl.akamaized.net/images/flagge/tiny/148.png?lm=1520611569" title="Schweiz"/></td>

Aufnehmender Verein:

In [17]:
soup.find_all('table')[2].find_all('tr')[1].find('td', {'class': 'no-border-rechts zentriert'}).a['title']

'FC Thun'

ID des aufnehmenden Vereins:

In [18]:
soup.find_all('table')[2].find_all('tr')[1].find('td', {'class': 'no-border-rechts zentriert'}).a['href'].split('/')[4]

'938'

Zielland:

In [19]:
soup.find_all('table')[2].find_all('tr')[1].find('td', {'class': 'no-border-links verein-flagge-transfer-cell'}).img['title']

'Schweiz'

Ablösesumme:

In [20]:
soup.find_all('table')[2].find_all('tr')[1].find_all('td', {'class': 'rechts'})[1].text

'ablösefrei'

### Der erste For-Loop-Versuch:

Nun erstelle ich mit einem For-Loop eine Liste aller Tabellen ('table'). Danach loope ich durch die Zeilen ('tr') jeder Tabelle, um an die jeweiligen Einträge ('td') zu kommen.

Zusatzbemerkung zu den Spielernamen: Ich beginne bei 1, weil ich die erste Tabelle nicht brauche und filtere die Tabellen mit Modulo, weil ich nur die Abgänge (alle Tabellen mit geraden Nummern), nicht aber die Zugänge brauche.

In [21]:
players_all = []
data_table = soup.find_all('table')

for i in range(1, len(data_table)):
        if i % 2 == 0:
            for j in range(len(data_table[i].find_all('tr'))):
                if (len(data_table[i].find_all('tr')[j].find_all('td')) > 0):
                    try:
                        season = soup.find('h1', {'class':'content-box-headline'}).text.strip().split(' ')[1]
                        name = data_table[i].find_all('tr')[j].find('td').find('div', {'class':'di nowrap'}).a['title']
                        name_id = data_table[i].find_all('tr')[j].find('td').find('div', {'class':'di nowrap'}).a['href'].split('/')[4]
                        alter = data_table[i].find_all('tr')[j].find('td', {'class': 'zentriert alter-transfer-cell'}).text
                        nationality = data_table[i].find_all('tr')[j].find('td', {'class': 'zentriert nat-transfer-cell'}).img['alt']
                        club1 = soup.find_all('h2', {'class':'content-box-headline content-box-headline--inverted content-box-headline--logo'})[(i // 2) - 1].img['alt']
                        id_club1 = soup.find_all('h2', {'class':''})[(i // 2) - 1].a['href'].split('-')[1]
                        club2 = data_table[i].find_all('tr')[j].find('td', {'class': 'no-border-rechts zentriert'}).a['title']
                        id_club2 = data_table[i].find_all('tr')[j].find('td', {'class': 'no-border-rechts zentriert'}).a['href'].split('/')[4]
                        target_land = data_table[i].find_all('tr')[j].find('td', {'class': 'no-border-links verein-flagge-transfer-cell'}).img['title']
                        fee = data_table[i].find_all('tr')[j].find_all('td', {'class': 'rechts'})[1].text
                
                               
                        dict = {'Saison': season,
                                'Name': name,
                                'Player_ID': name_id,
                                'Alter': alter,
                                'Nationalität': nationality,
                                'Abgebender Verein': club1,
                                'ID_1': id_club1,
                                'Aufnehmender Verein': club2,
                                'ID_2': id_club2,
                                'Zielland': target_land,
                                'Ablösesumme': fee}
                    
                        players_all.append(dict)
                    except:
                        pass                    

In [22]:
df = pd.DataFrame(players_all)

In [23]:
df

,Saison,Name,Player_ID,Alter,Nationalität,Abgebender Verein,ID_1,Aufnehmender Verein,ID_2,Zielland,Ablösesumme
0,03/04,Baykal Bellusci,2873,20,Türkei,Grasshopper Club Zürich,504,FC Thun,938,Schweiz,ablösefrei
1,03/04,José Manuel Chatruc,6574,27,Argentinien,Grasshopper Club Zürich,504,Barcelona SC Guayaquil,3523,Ecuador,ablösefrei
2,03/04,Sebastián Rozental,2868,26,Chile,Grasshopper Club Zürich,504,CD Universidad Católica,3277,Chile,ablösefrei
3,03/04,Mate Baturina,2869,29,Kroatien,Grasshopper Club Zürich,504,Bnei Yehuda Tel Aviv,9095,Israel,ablösefrei
4,03/04,Yassin Mikari,2878,20,Tunesien,Grasshopper Club Zürich,504,FC Luzern,434,Schweiz,ablösefrei
...,...,...,...,...,...,...,...,...,...,...,...
93,03/04,Olivier Boumelaha,81157,22,Frankreich,FC St. Gallen 1879,257,FC La Chaux-de-Fonds,2055,Schweiz,ablösefrei
94,03/04,Leo Lerinc,44938,28,Serbien,FC St. Gallen 1879,257,Ciudad de Murcia,4053,Spanien,?
95,03/04,Jan Berger,20105,26,Tschechien,FC St. Gallen 1879,257,FK Marila Pribram,2598,Tschechien,?
96,03/04,Ivan Dal Santo,9968,31,Schweiz,FC St. Gallen 1879,257,FC Zürich,260,Schweiz,?


## Der For-Loop für alle Saisons

Für die erste Transfer-Seite hat alles bestens geklappt, nun kommt der For-Loop über alle Saisons: Dafür erstelle ich zunächst eine Liste mit allen URLs und danach die zugehörigen Soups.

In [24]:
alle_saisons = range(2003,2023)

alle_urls = []

for saison in alle_saisons:
    alle_urls.append('https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=' + str(saison) + '&s_w=&leihe=0&intern=0')
alle_urls

['https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2003&s_w=&leihe=0&intern=0',
 'https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2004&s_w=&leihe=0&intern=0',
 'https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2005&s_w=&leihe=0&intern=0',
 'https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2006&s_w=&leihe=0&intern=0',
 'https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2007&s_w=&leihe=0&intern=0',
 'https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2008&s_w=&leihe=0&intern=0',
 'https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2009&s_w=&leihe=0&intern=0',
 'https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2010&s_w=&leihe=0&intern=0',
 'https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2011&s_w=&le

In [25]:
soups = []
for url in alle_urls:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
        req = urllib.request.Request(url = url, headers = headers)
        with urllib.request.urlopen(req) as response:
            page_html = response.read()
            soup = BeautifulSoup(page_html,"html.parser")
            soups.append(soup)
            print('working on', url)

working on https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2003&s_w=&leihe=0&intern=0
working on https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2004&s_w=&leihe=0&intern=0
working on https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2005&s_w=&leihe=0&intern=0
working on https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2006&s_w=&leihe=0&intern=0
working on https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2007&s_w=&leihe=0&intern=0
working on https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2008&s_w=&leihe=0&intern=0
working on https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2009&s_w=&leihe=0&intern=0
working on https://www.transfermarkt.de/super-league/transfers/wettbewerb/C1/plus/?saison_id=2010&s_w=&leihe=0&intern=0
working on https://www.transfermarkt.de/

Der For-Loop, der alle Informationen von allen Saions raussucht:

In [26]:
players_all = []
for soup in soups:
        data_table = soup.find_all('table')
        for i in range(1, len(data_table)):
                    if i % 2 == 0:
                        for j in range(len(data_table[i].find_all('tr'))):
                            if (len(data_table[i].find_all('tr')[j].find_all('td')) > 0):
                                try:
                                    season = soup.find('h1', {'class':'content-box-headline'}).text.strip().split(' ')[1]
                                    name = data_table[i].find_all('tr')[j].find('td').find('div', {'class':'di nowrap'}).a['title']
                                    name_id = data_table[i].find_all('tr')[j].find('td').find('div', {'class':'di nowrap'}).a['href'].split('/')[4]
                                    alter = data_table[i].find_all('tr')[j].find('td', {'class': 'zentriert alter-transfer-cell'}).text
                                    nationality = data_table[i].find_all('tr')[j].find('td', {'class': 'zentriert nat-transfer-cell'}).img['alt']
                                    club1 = soup.find_all('h2', {'class':'content-box-headline content-box-headline--inverted content-box-headline--logo'})[(i // 2) - 1].img['alt']
                                    id_club1 = soup.find_all('h2', {'class':''})[(i // 2) - 1].a['href'].split('-')[1]
                                    club2 = data_table[i].find_all('tr')[j].find('td', {'class': 'no-border-rechts zentriert'}).a['title']
                                    id_club2 = data_table[i].find_all('tr')[j].find('td', {'class': 'no-border-rechts zentriert'}).a['href'].split('/')[4]
                                    target_land = data_table[i].find_all('tr')[j].find('td', {'class': 'no-border-links verein-flagge-transfer-cell'}).img['title']
                                    fee = data_table[i].find_all('tr')[j].find_all('td', {'class': 'rechts'})[1].text


                                    dict = {'Saison': season,
                                            'Name': name,
                                            'Player_ID': name_id,
                                            'Alter': alter,
                                            'Nationalität': nationality,
                                            'Abgebender Verein': club1,
                                            'ID_1': id_club1,
                                            'Aufnehmender Verein': club2,
                                            'ID_2': id_club2,
                                            'Zielland': target_land,
                                            'Ablösesumme': fee}

                                    players_all.append(dict)
                                except Exception as e:
                                    print(name) #Zur Kontrolle: Das sind alle mit 'Karriereende', 'Unbekannt' oder 'Vereinslos'.
                                    pass 

Romain Crevoisier
Leandro D'Amico
Alex Da Silva
Dorjee Tsawa
Erich Hänzi
Adrian Lingenhag
Joël Descloux
Urs Fischer
Sébastien Jeanneret
Lucky Isibor
Bernardino Izael
Roger Küffer
Heinz Moser
Elvir Melunovic
Sascha Müller
Marco Pascolo
Sébastien Fournier
Aleksandar Bratic
Thomas Balmer
Ivan Stefanovic
Luiz Filho Jairo
Peter Kobel
Marek Citko
Cyrill Hasler
Oliver Tonelli
Martin Ogg
Marco Zwyssig
Mohamed Coubageat
Srdjan Maksimovic
Michael Hohl
Adrian Ilie
Mounir Soufiani
Henri Siqueira-Barras
Florent Delay
Salvatore Caló
Stefan Wolf
Murat Yakin
Thomas Mandl
Silvan Aegerter
André Muff
Delfim
Joël Magnin
Artur Petrosyan
Leonardo Oliveira
Simon Leu
Flavio Agosti
Remo Pesenti
Dorjee Tsawa
Benoît Cauet
Luis Calapes
Emanuele Di Zenzo
Adel Chedli
Fabrice Borer
João Manuel Pinto
Stéphane Sarni
Yao Aziawonou
Jordan Simpson
Dominique Aebi
Dusan Pavlovic
Sehid Sinani
Thuram
Johan Berisha
Krešimir Stanic
Papa Malick Ba
Bastien Geiger
Thomas Häberli
Faysal El Idrissi
Sven Christ
Carlos Alberto
Aymen 

In [27]:
df = pd.DataFrame.from_dict(players_all)

In [28]:
df

,Saison,Name,Player_ID,Alter,Nationalität,Abgebender Verein,ID_1,Aufnehmender Verein,ID_2,Zielland,Ablösesumme
0,03/04,Baykal Bellusci,2873,20,Türkei,Grasshopper Club Zürich,504,FC Thun,938,Schweiz,ablösefrei
1,03/04,José Manuel Chatruc,6574,27,Argentinien,Grasshopper Club Zürich,504,Barcelona SC Guayaquil,3523,Ecuador,ablösefrei
2,03/04,Sebastián Rozental,2868,26,Chile,Grasshopper Club Zürich,504,CD Universidad Católica,3277,Chile,ablösefrei
3,03/04,Mate Baturina,2869,29,Kroatien,Grasshopper Club Zürich,504,Bnei Yehuda Tel Aviv,9095,Israel,ablösefrei
4,03/04,Yassin Mikari,2878,20,Tunesien,Grasshopper Club Zürich,504,FC Luzern,434,Schweiz,ablösefrei
...,...,...,...,...,...,...,...,...,...,...,...
1741,22/23,Silvan Sidler,342063,23,Schweiz,FC Luzern,434,Arminia Bielefeld,10,Deutschland,ablösefrei
1742,22/23,Marvin Schulz,168678,27,Deutschland,FC Luzern,434,Holstein Kiel,269,Deutschland,ablösefrei
1743,22/23,Salah Aziz Binous,511143,21,Schweiz,FC Luzern,434,SC YF Juventus Zürich,2395,Schweiz,?
1744,22/23,Ashvin Balaruban,507454,20,Schweiz,FC Luzern,434,Neuchâtel Xamax FCS,625,Schweiz,?


Ich speichere das erste Resultat als CSV ab und mache ein paar erste Abfragen:

In [29]:
df.to_csv('Alle_Abgänge_Super_League.csv')

In [30]:
len(df[df['Zielland'] != 'Schweiz'])

833

In [31]:
len(df[df['Zielland'] == 'Schweiz'])

913

In [32]:
len(df[df['Zielland'] == 'Deutschland'])

135

In [33]:
len(df[df['Zielland'] == 'England'])

44

In [34]:
len(df[df['Zielland'] == 'Italien'])

68

In [35]:
len(df[df['Zielland'] == 'Spanien'])

36

In [36]:
len(df[df['Zielland'] == 'Frankreich'])

83

In [37]:
#Ich checke kurz, ob der neuste Super-League-Abgang drin ist.
df[df['Name'] == 'Filip Stojilkovic']

,Saison,Name,Player_ID,Alter,Nationalität,Abgebender Verein,ID_1,Aufnehmender Verein,ID_2,Zielland,Ablösesumme
1711,22/23,Filip Stojilkovic,464195,23,Schweiz,FC Sion,321,SV Darmstadt 98,105,Deutschland,?


## Liga-Zugehörigkeit der Klubs

Für die Klubs aus England, Deutschland, Italien, Frankreich und Spanien muss sich zwingend wissen, in welcher Liga der abnehmende Verein zum Zeitpunkt des Transfers gespielt hat. Das kann ich auf der folgenden Basis-Seite scrapen: 'https://www.transfermarkt.de/-/startseite/verein/' + 'ID_2'

Mein Vorgehen: Ich erstelle eine Liste mit allen Liga-Zughörigkeiten pro Klub für jedes Jahr seit 2003. Dafür erstelle ich zunächst eine Liste mit allen Klubs, die ich benötige.

In [38]:
club_ids =[]
club_ids = list(df['ID_2'])
len(club_ids)

1746

In [39]:
unique_club_ids = set(club_ids)
unique_club_ids = (list(unique_club_ids))
len(unique_club_ids)

665

Ich erstelle eine Liste mit allen URLs der Klubseiten «Historische Platzierungen»:

In [40]:
all_urls = []
for club_id in unique_club_ids:
    url = 'https://www.transfermarkt.de/-/platzierungen/verein/' + club_id
    all_urls.append(url)
len(all_urls)

665

### Ich spiele wieder alles anhand einer Seite durch:

Ich erstelle eine Soup für einen Klub, um nach den benötigten Einträgen zu suchen.

In [41]:
my_url = 'https://www.transfermarkt.de/-/platzierungen/verein/79'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
req = urllib.request.Request(url = my_url, headers = headers)
with urllib.request.urlopen(req) as response:
   page_html = response.read()
my_soup = BeautifulSoup(page_html,"html.parser")

So komme ich an diese Saison ran:

In [42]:
my_soup.find_all('table')[1].find_all('tr')[1].find_all('td', {'class': 'zentriert'})[0].text

'22/23'

So komme ich an die Ligahöhe ran:

In [43]:
my_soup.find_all('table')[1].find_all('tr')[1].find_all('td', {'class': 'zentriert'})[2].text

'1.Liga'

So an den Verein:

In [44]:
my_soup.find('div', {'class':'data-header__profile-container'}).img['alt']

'VfB Stuttgart'

So an die Club_ID:

In [45]:
my_soup.find('div', {'class': 'quick-select-wrapper'}).find('tm-quick-select-bar')['default-club']

'79'

Das ist die Länge der Tabelle 1 mit mit den jährlichen Einträgen der Ligazugehörigkeit (VfB Stuttgart):

In [46]:
len(my_soup.find_all('table')[1].find_all('tr'))

65

### Jetzt die weiteren For-Loops

Ich erstelle alle Soups:

In [47]:
soups = []
for url in all_urls:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
    req = urllib.request.Request(url = url, headers = headers)
    with urllib.request.urlopen(req) as response:
        page_html = response.read()
        soup = BeautifulSoup(page_html,"html.parser")
        soups.append(soup)
        print('working on', url)

working on https://www.transfermarkt.de/-/platzierungen/verein/3841
working on https://www.transfermarkt.de/-/platzierungen/verein/2411
working on https://www.transfermarkt.de/-/platzierungen/verein/13
working on https://www.transfermarkt.de/-/platzierungen/verein/18
working on https://www.transfermarkt.de/-/platzierungen/verein/862
working on https://www.transfermarkt.de/-/platzierungen/verein/618
working on https://www.transfermarkt.de/-/platzierungen/verein/6321
working on https://www.transfermarkt.de/-/platzierungen/verein/51724
working on https://www.transfermarkt.de/-/platzierungen/verein/2417
working on https://www.transfermarkt.de/-/platzierungen/verein/159
working on https://www.transfermarkt.de/-/platzierungen/verein/10640
working on https://www.transfermarkt.de/-/platzierungen/verein/80
working on https://www.transfermarkt.de/-/platzierungen/verein/208
working on https://www.transfermarkt.de/-/platzierungen/verein/6297
working on https://www.transfermarkt.de/-/platzierungen/

working on https://www.transfermarkt.de/-/platzierungen/verein/37820
working on https://www.transfermarkt.de/-/platzierungen/verein/2293
working on https://www.transfermarkt.de/-/platzierungen/verein/1292
working on https://www.transfermarkt.de/-/platzierungen/verein/2049
working on https://www.transfermarkt.de/-/platzierungen/verein/413
working on https://www.transfermarkt.de/-/platzierungen/verein/36811
working on https://www.transfermarkt.de/-/platzierungen/verein/141
working on https://www.transfermarkt.de/-/platzierungen/verein/168
working on https://www.transfermarkt.de/-/platzierungen/verein/4053
working on https://www.transfermarkt.de/-/platzierungen/verein/408
working on https://www.transfermarkt.de/-/platzierungen/verein/7732
working on https://www.transfermarkt.de/-/platzierungen/verein/13959
working on https://www.transfermarkt.de/-/platzierungen/verein/36
working on https://www.transfermarkt.de/-/platzierungen/verein/626
working on https://www.transfermarkt.de/-/platzierun

working on https://www.transfermarkt.de/-/platzierungen/verein/9057
working on https://www.transfermarkt.de/-/platzierungen/verein/1763
working on https://www.transfermarkt.de/-/platzierungen/verein/991
working on https://www.transfermarkt.de/-/platzierungen/verein/4032
working on https://www.transfermarkt.de/-/platzierungen/verein/68608
working on https://www.transfermarkt.de/-/platzierungen/verein/52921
working on https://www.transfermarkt.de/-/platzierungen/verein/91
working on https://www.transfermarkt.de/-/platzierungen/verein/716
working on https://www.transfermarkt.de/-/platzierungen/verein/3008
working on https://www.transfermarkt.de/-/platzierungen/verein/332
working on https://www.transfermarkt.de/-/platzierungen/verein/242
working on https://www.transfermarkt.de/-/platzierungen/verein/398
working on https://www.transfermarkt.de/-/platzierungen/verein/3523
working on https://www.transfermarkt.de/-/platzierungen/verein/2833
working on https://www.transfermarkt.de/-/platzierung

working on https://www.transfermarkt.de/-/platzierungen/verein/2969
working on https://www.transfermarkt.de/-/platzierungen/verein/2282
working on https://www.transfermarkt.de/-/platzierungen/verein/257
working on https://www.transfermarkt.de/-/platzierungen/verein/124
working on https://www.transfermarkt.de/-/platzierungen/verein/20375
working on https://www.transfermarkt.de/-/platzierungen/verein/58
working on https://www.transfermarkt.de/-/platzierungen/verein/786
working on https://www.transfermarkt.de/-/platzierungen/verein/1264
working on https://www.transfermarkt.de/-/platzierungen/verein/14470
working on https://www.transfermarkt.de/-/platzierungen/verein/15
working on https://www.transfermarkt.de/-/platzierungen/verein/3500
working on https://www.transfermarkt.de/-/platzierungen/verein/24032
working on https://www.transfermarkt.de/-/platzierungen/verein/3785
working on https://www.transfermarkt.de/-/platzierungen/verein/7084
working on https://www.transfermarkt.de/-/platzierun

working on https://www.transfermarkt.de/-/platzierungen/verein/150
working on https://www.transfermarkt.de/-/platzierungen/verein/3197
working on https://www.transfermarkt.de/-/platzierungen/verein/1910
working on https://www.transfermarkt.de/-/platzierungen/verein/941
working on https://www.transfermarkt.de/-/platzierungen/verein/499
working on https://www.transfermarkt.de/-/platzierungen/verein/1918
working on https://www.transfermarkt.de/-/platzierungen/verein/584
working on https://www.transfermarkt.de/-/platzierungen/verein/500
working on https://www.transfermarkt.de/-/platzierungen/verein/221
working on https://www.transfermarkt.de/-/platzierungen/verein/833
working on https://www.transfermarkt.de/-/platzierungen/verein/3911
working on https://www.transfermarkt.de/-/platzierungen/verein/496
working on https://www.transfermarkt.de/-/platzierungen/verein/8841
working on https://www.transfermarkt.de/-/platzierungen/verein/18234
working on https://www.transfermarkt.de/-/platzierungen

working on https://www.transfermarkt.de/-/platzierungen/verein/2375
working on https://www.transfermarkt.de/-/platzierungen/verein/24898
working on https://www.transfermarkt.de/-/platzierungen/verein/3508
working on https://www.transfermarkt.de/-/platzierungen/verein/26789
working on https://www.transfermarkt.de/-/platzierungen/verein/35
working on https://www.transfermarkt.de/-/platzierungen/verein/385
working on https://www.transfermarkt.de/-/platzierungen/verein/32704
working on https://www.transfermarkt.de/-/platzierungen/verein/5259
working on https://www.transfermarkt.de/-/platzierungen/verein/845
working on https://www.transfermarkt.de/-/platzierungen/verein/65
working on https://www.transfermarkt.de/-/platzierungen/verein/3205
working on https://www.transfermarkt.de/-/platzierungen/verein/951
working on https://www.transfermarkt.de/-/platzierungen/verein/4294
working on https://www.transfermarkt.de/-/platzierungen/verein/5220
working on https://www.transfermarkt.de/-/platzierun

Nun greife ich die oben definierten, benötigten Daten ab:

In [48]:
clubs_all = []
for soup in soups:
    if (len(soup.find_all('table')) < 2):
        continue
    for i in range(1, len(soup.find_all('table')[1].find_all('tr'))):
        club = soup.find('div', {'class':'data-header__profile-container'}).img['alt']
        club_id = soup.find('div', {'class': 'quick-select-wrapper'}).find('tm-quick-select-bar')['default-club']
        season = soup.find_all('table')[1].find_all('tr')[i].find_all('td', {'class': 'zentriert'})[0].text
        league = soup.find_all('table')[1].find_all('tr')[i].find_all('td', {'class': 'zentriert'})[2].text
            
        dict = {'Aufnehmender Verein': club,
                'ID_2': club_id,
                'Saison': season,
                'Ligahöhe': league}
            
        clubs_all.append(dict)

Das Dataframe mit den Ligahöhen aller Klubs:

In [49]:
df_ligahoehe = pd.DataFrame.from_dict(clubs_all)
df_ligahoehe

,Aufnehmender Verein,ID_2,Saison,Ligahöhe
0,Istanbul Güngörenspor,3841,14/15,4.Liga
1,Istanbul Güngörenspor,3841,13/14,3.Liga
2,Istanbul Güngörenspor,3841,12/13,3.Liga
3,Istanbul Güngörenspor,3841,11/12,2.Liga
4,Istanbul Güngörenspor,3841,10/11,2.Liga
...,...,...,...,...
16942,FC Progrès Niederkorn,12346,08/09,1.Liga
16943,FC Progrès Niederkorn,12346,07/08,1.Liga
16944,FC Progrès Niederkorn,12346,06/07,1.Liga
16945,FC Progrès Niederkorn,12346,02/03,1.Liga


Speichern des Dataframes als CSV plus ein paar Anpassungen.

In [50]:
df_ligahoehe.to_csv('Alle_Ligahoehen_der_aufnehmenden_Klubs.csv')
df_ligahoehe['ID_2'] = df_ligahoehe['ID_2'].astype('int')
df_ligahoehe.dtypes

Aufnehmender Verein    object
ID_2                    int32
Saison                 object
Ligahöhe               object
dtype: object

## Mergen der beiden Dataframes


Ich muss nun die beiden erstellten Dataframes auf zwei Konditionen zusammenführen: ID_2 und Saison. Danach weiss ich, in welcher Liga der aufnehmende Verein zum Transferzeitpunkt gespielt hat.

In [51]:
df_transfers = pd.read_csv('Alle_Abgänge_Super_League.csv')

In [52]:
del df_transfers['Unnamed: 0']

In [53]:
df_transfers.dtypes

Saison                 object
Name                   object
Player_ID               int64
Alter                  object
Nationalität           object
Abgebender Verein      object
ID_1                    int64
Aufnehmender Verein    object
ID_2                    int64
Zielland               object
Ablösesumme            object
dtype: object

In [54]:
df_new_transfers = pd.merge(df_transfers, df_ligahoehe,  how='left', left_on=['Saison','ID_2'], right_on = ['Saison','ID_2'])
df_new_transfers

,Saison,Name,Player_ID,Alter,Nationalität,Abgebender Verein,ID_1,Aufnehmender Verein_x,ID_2,Zielland,Ablösesumme,Aufnehmender Verein_y,Ligahöhe
0,03/04,Baykal Bellusci,2873,20,Türkei,Grasshopper Club Zürich,504,FC Thun,938,Schweiz,ablösefrei,FC Thun,1.Liga
1,03/04,José Manuel Chatruc,6574,27,Argentinien,Grasshopper Club Zürich,504,Barcelona SC Guayaquil,3523,Ecuador,ablösefrei,NaN,NaN
2,03/04,Sebastián Rozental,2868,26,Chile,Grasshopper Club Zürich,504,CD Universidad Católica,3277,Chile,ablösefrei,NaN,NaN
3,03/04,Mate Baturina,2869,29,Kroatien,Grasshopper Club Zürich,504,Bnei Yehuda Tel Aviv,9095,Israel,ablösefrei,NaN,NaN
4,03/04,Yassin Mikari,2878,20,Tunesien,Grasshopper Club Zürich,504,FC Luzern,434,Schweiz,ablösefrei,FC Luzern,2.Liga
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1747,22/23,Silvan Sidler,342063,23,Schweiz,FC Luzern,434,Arminia Bielefeld,10,Deutschland,ablösefrei,Arminia Bielefeld,2.Liga
1748,22/23,Marvin Schulz,168678,27,Deutschland,FC Luzern,434,Holstein Kiel,269,Deutschland,ablösefrei,Holstein Kiel,2.Liga
1749,22/23,Salah Aziz Binous,511143,21,Schweiz,FC Luzern,434,SC YF Juventus Zürich,2395,Schweiz,?,SC YF Juventus Zürich,3.Liga
1750,22/23,Ashvin Balaruban,507454,20,Schweiz,FC Luzern,434,Neuchâtel Xamax FCS,625,Schweiz,?,Neuchâtel Xamax FCS,2.Liga


Nicht bei allen Transfers hat das geklappt, die NaNs sind aber kein Problem, da sie ausschliesslich tiefer spielende und «exotische» Vereine betreffen. Für die Top-5-Ligen hat es wunderbar funktioniert.

In [55]:
del df_new_transfers['Aufnehmender Verein_y']
df_new_transfers.rename(columns = {'Aufnehmender Verein_x':'Aufnehmender Verein'}, inplace = True)

In [56]:
#Ablösesummen korrigieren:
df_new_transfers['Ablösesumme'] = df_new_transfers['Ablösesumme'].replace({'€':'',' Tsd.':'000', ',':'', ' Mio.':'0000'}, regex=True)
df_new_transfers['Ablösesumme'] = df_new_transfers['Ablösesumme'].replace({'ablösefrei': '0', '?':'0', '-':'0'})

In [57]:
df_new_transfers['Ablösesumme'] = pd.to_numeric(df_new_transfers['Ablösesumme'])

In [58]:
#Bei einem einzigen Spieler fehlt das Alter. Dieses füge ich manuell ein.
df_new_transfers.loc[13, 'Alter'] = 20
df_new_transfers.loc[214, 'Alter'] = 21
df_new_transfers[df_new_transfers['Name'] == 'Fabio Digenti']
df_new_transfers['Alter'] = df_new_transfers['Alter'].astype(str).astype(int)

In [59]:
df_new_transfers.dtypes

Saison                  object
Name                    object
Player_ID                int64
Alter                    int32
Nationalität            object
Abgebender Verein       object
ID_1                     int64
Aufnehmender Verein     object
ID_2                     int64
Zielland                object
Ablösesumme            float64
Ligahöhe                object
dtype: object

In [60]:
df_new_transfers.to_csv('Alle_Transfers_Final.csv')

## Die wichtigsten Abfragen:

In [61]:
#Anazahl Transfers gesamt:
len(df_new_transfers)

1752

In [62]:
#So viele verschiedene Zielländer gibt es:
df_new_transfers['Zielland'].nunique()

73

In [63]:
#Anzahl Transfers pro Zielland:
df_new_transfers['Zielland'].value_counts()

Schweiz                913
Deutschland            135
Frankreich              83
Italien                 68
England                 44
                      ... 
Belarus                  1
Libyen                   1
Nigeria                  1
Trinidad und Tobago      1
Réunion                  1
Name: Zielland, Length: 73, dtype: int64

In [64]:
df_new_transfers['Zielland'].value_counts().to_csv('Transfers_per Zielland.csv')

In [65]:
len(df_new_transfers[df_new_transfers['Zielland'] != 'Schweiz'])

839

In [66]:
len(df_new_transfers[df_new_transfers['Zielland'] == 'Frankreich'])

83

In [67]:
df_tranfers_ausland = df_new_transfers[df_new_transfers['Zielland'] != 'Schweiz']

In [68]:
df_new_transfers.groupby('Zielland')['Ligahöhe'].value_counts().to_csv('Anzahl_Transfers_Ligahoehe_und_Land.csv')

In [69]:
#Anzahl Transfers pro Top-5-Liga:
df_PremierLeague = df_new_transfers[(df_new_transfers['Zielland'] == 'England') & (df_new_transfers['Ligahöhe'] == '1.Liga')]
df_SerieA = df_new_transfers[(df_new_transfers['Zielland'] == 'Italien') & (df_new_transfers['Ligahöhe'] == '1.Liga')]
df_LaLiga = df_new_transfers[(df_new_transfers['Zielland'] == 'Spanien') & (df_new_transfers['Ligahöhe'] == '1.Liga')]
df_Bundesliga = df_new_transfers[(df_new_transfers['Zielland'] == 'Deutschland') & (df_new_transfers['Ligahöhe'] == '1.Liga')]
df_Ligue1 = df_new_transfers[(df_new_transfers['Zielland'] == 'Frankreich') & (df_new_transfers['Ligahöhe'] == '1.Liga')]

In [70]:
TopFiveTransfers = len(df_PremierLeague) + len(df_SerieA) + len(df_LaLiga) + len(df_Bundesliga) + len(df_Ligue1)
TopFiveTransfers

186

In [71]:
len(df_PremierLeague)

25

In [72]:
#Dataframe aller Transfers in eine Top-5-Liga:
df_Topfive = pd.concat([df_PremierLeague, df_SerieA, df_LaLiga, df_Bundesliga, df_Ligue1])
df_Topfive.to_csv('Alle-Top-Fünf-Transfers.csv')
df_Topfive

,Saison,Name,Player_ID,Alter,Nationalität,Abgebender Verein,ID_1,Aufnehmender Verein,ID_2,Zielland,Ablösesumme,Ligahöhe
52,03/04,Philippe Senderos,4277,18,Schweiz,Servette FC,61,FC Arsenal,11,England,3550000.0,1.Liga
98,04/05,Thimothée Atouba,2887,22,Kamerun,FC Basel 1893,26,Tottenham Hotspur,148,England,4500000.0,1.Liga
169,04/05,Reto Ziegler,9915,18,Schweiz,Grasshopper Club Zürich,504,Tottenham Hotspur,148,England,1000000.0,1.Liga
331,06/07,Bruno Berner,1589,29,Schweiz,FC Basel 1893,26,Blackburn Rovers,164,England,500000.0,1.Liga
439,07/08,Blerim Dzemaili,9967,21,Schweiz,FC Zürich,260,Bolton Wanderers,355,England,0.0,1.Liga
...,...,...,...,...,...,...,...,...,...,...,...,...
1600,21/22,Grejohn Kyei,321647,26,Frankreich,Servette FC,61,Clermont Foot 63,3524,Frankreich,0.0,1.Liga
1602,21/22,Arial Mendy,422715,26,Senegal,Servette FC,61,Clermont Foot 63,3524,Frankreich,0.0,1.Liga
1621,21/22,Moritz Jenz,460245,22,Deutschland,FC Lausanne-Sport,527,FC Lorient,1158,Frankreich,3500000.0,1.Liga
1626,21/22,Gabriel Barès,484743,21,Schweiz,FC Lausanne-Sport,527,Montpellier HSC,969,Frankreich,0.0,1.Liga


In [73]:
#Dataframe aller Transfers in die deutsche Bundesliga
df_Topfive[df_Topfive['Zielland'] == 'Deutschland']

,Saison,Name,Player_ID,Alter,Nationalität,Abgebender Verein,ID_1,Aufnehmender Verein,ID_2,Zielland,Ablösesumme,Ligahöhe
14,03/04,Marco Streller,4879,22,Schweiz,FC Basel 1893,26,VfB Stuttgart,79,Deutschland,3000000.0,1.Liga
15,03/04,Hakan Yakin,1054,26,Schweiz,FC Basel 1893,26,VfB Stuttgart,79,Deutschland,2500000.0,1.Liga
99,04/05,Mario Cantaluppi,2807,30,Schweiz,FC Basel 1893,26,1.FC Nürnberg,4,Deutschland,650000.0,1.Liga
105,04/05,Leandro,499,29,Brasilien,BSC Young Boys,452,Hannover 96,42,Deutschland,450000.0,1.Liga
143,04/05,Tranquillo Barnetta,6645,19,Schweiz,FC St. Gallen 1879,257,Bayer 04 Leverkusen,15,Deutschland,250000.0,1.Liga
...,...,...,...,...,...,...,...,...,...,...,...,...
1518,20/21,Omar Alderete,353032,23,Paraguay,FC Basel 1893,26,Hertha BSC,44,Deutschland,4000000.0,1.Liga
1583,21/22,Silvan Widmer,168989,28,Schweiz,FC Basel 1893,26,1.FSV Mainz 05,39,Deutschland,2500000.0,1.Liga
1586,21/22,Afimico Pululu,410649,22,Angola,FC Basel 1893,26,SpVgg Greuther Fürth,65,Deutschland,0.0,1.Liga
1670,22/23,Jordan,355369,26,Vereinigte Staaten,BSC Young Boys,452,1.FC Union Berlin,89,Deutschland,6000000.0,1.Liga


In [74]:
df_t5pl = df_Topfive.groupby('Saison')['Zielland'].value_counts()
df_t5pl.tail(10)

Saison  Zielland   
20/21   England        1
21/22   Frankreich     5
        Deutschland    2
        Italien        2
        England        1
        Spanien        1
22/23   Italien        3
        Deutschland    2
        England        1
        Frankreich     1
Name: Zielland, dtype: int64

In [75]:
#Das Durchschnittsalter aller Spieler bei ihrem Wechsel
df_new_transfers['Alter'].mean()

25.08732876712329

In [76]:
#Das Durchschnittsalter aller Spieler bei ihrem Wechsel ins Ausland
df_tranfers_ausland['Alter'].mean()

25.206197854588797

In [77]:
#Das Durchschnittsalter aller Spieler beim ihrem Wechsel in eine Top-5-Liga
df_Topfive['Alter'].mean()

23.5

In [78]:
#Von welchen Klubs wechseln Spieler in eine Top-5-Liga?
df_Topfive['Abgebender Verein'].value_counts().to_csv('Klubs_mit-Top-5-Abgängen.csv')
df_Topfive['Abgebender Verein'].value_counts()

FC Basel 1893              57
BSC Young Boys             30
Grasshopper Club Zürich    25
FC Zürich                  23
FC Sion                    12
Servette FC                 9
FC St. Gallen 1879          6
FC Luzern                   6
FC Lausanne-Sport           4
FC Aarau                    3
Neuchâtel Xamax FCS         3
Neuchâtel Xamax             2
AC Bellinzona               2
FC Lugano                   2
FC Thun                     1
FC Vaduz                    1
Name: Abgebender Verein, dtype: int64

In [79]:
#Die 20 höchsten Ablösesummen:
df_new_transfers['Ablösesumme'].sort_values(ascending=False).head(20)

1164    26500000.0
1248    21500000.0
1346    18000000.0
926     16500000.0
1581    15500000.0
1387    15000000.0
1072    12500000.0
1328    12000000.0
856     11800000.0
1418    11300000.0
1419    11200000.0
707     11000000.0
927     10000000.0
1491    10000000.0
1073     9400000.0
1492     9200000.0
1004     9000000.0
759      8500000.0
857      8500000.0
1407     8000000.0
Name: Ablösesumme, dtype: float64

In [80]:
df_new_transfers[df_new_transfers['Ablösesumme'] >= 5000000]

,Saison,Name,Player_ID,Alter,Nationalität,Abgebender Verein,ID_1,Aufnehmender Verein,ID_2,Zielland,Ablösesumme,Ligahöhe
327,06/07,Matías Delgado,8886,23,Argentinien,FC Basel 1893,26,Besiktas JK,114,Türkei,5500000.0,1.Liga
440,07/08,Felipe Caicedo,37834,19,Ecuador,FC Basel 1893,26,Manchester City,281,England,7000000.0,1.Liga
441,07/08,Ivan Rakitic,32467,19,Kroatien,FC Basel 1893,26,FC Schalke 04,33,Deutschland,5000000.0,1.Liga
448,07/08,Gelson Fernandes,20081,20,Schweiz,FC Sion,321,Manchester City,281,England,5500000.0,1.Liga
679,10/11,Samuel Inkoom,106306,21,Ghana,FC Basel 1893,26,Dnipro Dnipropetrovsk (-2020),339,Ukraine,5300000.0,1.Liga
707,10/11,Seydou Doumbia,75287,22,Elfenbeinküste,BSC Young Boys,452,ZSKA Moskau,2410,Russland,11000000.0,1.Liga
759,11/12,Ricardo Rodríguez,86784,19,Schweiz,FC Zürich,260,VfL Wolfsburg,82,Deutschland,8500000.0,1.Liga
777,11/12,Giovanni Sio,76293,22,Elfenbeinküste,FC Sion,321,VfL Wolfsburg,82,Deutschland,5800000.0,1.Liga
856,12/13,Xherdan Shaqiri,86792,20,Schweiz,FC Basel 1893,26,FC Bayern München,27,Deutschland,11800000.0,1.Liga
857,12/13,Granit Xhaka,111455,19,Schweiz,FC Basel 1893,26,Borussia Mönchengladbach,18,Deutschland,8500000.0,1.Liga
